# 🌼 Transfer Learning com VGG16 no Oxford Flowers 102 (Google Colab)

In [ ]:
# TRANSFER LEARNING E FINE-TUNING COM VGG16 - OXFORD FLOWERS 102

## 🔁 1. Instalação de dependências
# Instala as bibliotecas TensorFlow e TensorFlow Datasets.
# -q: Garante uma instalação "silenciosa", sem exibir muitas mensagens no terminal.
!pip install tensorflow tensorflow_datasets -q

## 📥 2. Carregando o Oxford Flowers 102
import tensorflow_datasets as tfds # Importa a biblioteca para carregar datasets do TensorFlow

# Carrega o dataset 'oxford_flowers102'.
# with_info=True: Retorna metadados do dataset (como número de classes, etc.).
# as_supervised=True: Retorna o dataset no formato (imagem, rótulo).
dataset, info = tfds.load('oxford_flowers102', with_info=True, as_supervised=True)

# Divide o dataset nas partes de treino, validação e teste.
train_ds, val_ds, test_ds = dataset['train'], dataset['validation'], dataset['test']

# Obtém o número total de classes de flores a partir dos metadados do dataset.
num_classes = info.features['label'].num_classes

## 🧹 3. Pré-processamento (tamanho e normalização)
import tensorflow as tf # Importa o TensorFlow para operações de manipulação de imagem

IMG_SIZE = 224 # Define o tamanho que as imagens terão (224x224 pixels), padrão para VGG16.
BATCH_SIZE = 32 # Define o número de imagens processadas por lote (batch) durante o treinamento.

# Função para formatar as imagens e rótulos.
def format_image(image, label):
    # Redimensiona a imagem para o tamanho desejado (224x224).
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    # Converte os valores dos pixels para float32 e os normaliza para o intervalo [0, 1].
    # (Pixels originais são de 0-255, a normalização ajuda a rede neural a aprender melhor).
    image = tf.cast(image, tf.float32) / 255.0
    # Converte o rótulo numérico para um vetor one-hot encoded (ex: 5 -> [0,0,0,0,1,0,...]).
    # Isso é necessário para a função de perda 'categorical_crossentropy'.
    return image, tf.one_hot(label, num_classes)

# Aplica a função de pré-processamento aos datasets e configura o pipeline de dados.
# .map(format_image): Aplica a função format_image a cada elemento do dataset.
# .shuffle(1000): Embaralha o dataset de treino para garantir que os lotes sejam diversos. O número 1000 é o tamanho do buffer de embaralhamento.
# .batch(BATCH_SIZE): Agrupa os elementos em lotes do tamanho especificado.
# .prefetch(1): Permite que os dados sejam pré-carregados enquanto o modelo está treinando, otimizando a performance.
train_ds = train_ds.map(format_image).shuffle(1000).batch(BATCH_SIZE).prefetch(1)
val_ds = val_ds.map(format_image).batch(BATCH_SIZE).prefetch(1)
test_ds = test_ds.map(format_image).batch(BATCH_SIZE).prefetch(1)

## 🧠 4. Modelo com Transfer Learning (VGG16)
# Importa as classes necessárias do Keras.
from tensorflow.keras.applications import VGG16 # Para carregar o modelo VGG16 pré-treinado.
from tensorflow.keras.models import Model # Para criar um modelo funcional a partir de camadas.
from tensorflow.keras.layers import Flatten, Dense, Dropout, Input # Camadas comuns para redes neurais.
from tensorflow.keras.optimizers import Adam # Otimizador Adam para ajustar os pesos do modelo.

# Carrega o modelo VGG16 pré-treinado no dataset ImageNet.
# weights='imagenet': Usa os pesos aprendidos no ImageNet.
# include_top=False: Exclui as camadas totalmente conectadas finais (cabeçalho de classificação) do modelo original.
# input_shape: Define a forma de entrada esperada (altura, largura, canais de cor).
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Congela todas as camadas do modelo base VGG16.
# Isso significa que os pesos dessas camadas NÃO serão atualizados durante o treinamento inicial.
# Estamos usando a VGG16 como um extrator de características fixo.
base_model.trainable = False

# Constrói o novo cabeçalho de classificação que será adicionado ao modelo base.
# Flatten(): Transforma a saída 3D da VGG16 (mapas de características) em um vetor 1D.
x = Flatten()(base_model.output)
# Dense(256, activation='relu'): Uma camada totalmente conectada com 256 neurônios e função de ativação ReLU.
x = Dense(256, activation='relu')(x)
# Dropout(0.5): Desativa aleatoriamente 50% dos neurônios durante o treinamento para evitar overfitting.
x = Dropout(0.5)(x)
# Dense(num_classes, activation='softmax'): A camada de saída, com 'num_classes' neurônios.
# 'softmax' é usada para classificação multiclasse, resultando em probabilidades para cada classe.
output = Dense(num_classes, activation='softmax')(x)

# Cria o modelo final combinando a VGG16 (congelada) com o novo cabeçalho de classificação.
model = Model(inputs=base_model.input, outputs=output)

# Compila o modelo.
# optimizer=Adam(): O otimizador Adam é usado para ajustar os pesos das camadas treináveis.
# loss='categorical_crossentropy': Função de perda para problemas de classificação multiclasse com rótulos one-hot.
# metrics=['accuracy']: Monitora a acurácia (precisão) durante o treinamento.
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

print("Treinando modelo com Transfer Learning (camadas congeladas)...")
# Treina o modelo.
# train_ds: Dataset de treino.
# validation_data=val_ds: Dataset de validação para monitorar o desempenho em dados não vistos.
# epochs=10: Número de vezes que o modelo irá percorrer todo o dataset de treino.
history_tl = model.fit(train_ds, validation_data=val_ds, epochs=10)

## 🔓 5. Fine-Tuning (descongelando camadas finais)
# Descongela todas as camadas do modelo base para prepará-las para o fine-tuning.
base_model.trainable = True

# Congela novamente todas as camadas do modelo base, EXCETO as últimas 4.
# A ideia é que as camadas mais profundas (finais) do VGG16 possam ser ajustadas aos dados específicos das flores,
# enquanto as camadas iniciais (que capturam características genéricas) permanecem fixas.
for layer in base_model.layers[:-4]:
    layer.trainable = False

# Recompila o modelo com uma taxa de aprendizado muito menor para o fine-tuning.
# Uma taxa de aprendizado menor evita "desaprender" o conhecimento pré-existente e permite ajustes finos.
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

print("Treinando modelo com Fine-Tuning (últimas camadas descongeladas)...")
# Continua o treinamento do modelo por mais 5 épocas.
# Agora, tanto o cabeçalho de classificação quanto as últimas 4 camadas da VGG16 serão ajustados.
history_ft = model.fit(train_ds, validation_data=val_ds, epochs=5)

## 📊 6. Avaliação no conjunto de teste
# Avalia o desempenho final do modelo no conjunto de teste (dados totalmente novos).
# Isso dá uma estimativa realista de como o modelo se comportará em dados do mundo real.
loss, acc = model.evaluate(test_ds)
print(f"Acurácia final no conjunto de teste: {acc:.4f}")

## 📈 7. Gráficos comparativos
import matplotlib.pyplot as plt # Importa a biblioteca para criar gráficos

# Função para plotar os históricos de acurácia de treino e validação.
def plot_history(hist1, hist2, title):
    # Combina os históricos de acurácia de validação das duas fases (transfer learning e fine-tuning).
    plt.plot(hist1.history['val_accuracy'] + hist2.history['val_accuracy'], label='Val Accuracy')
    # Combina os históricos de acurácia de treino das duas fases.
    plt.plot(hist1.history['accuracy'] + hist2.history['accuracy'], label='Train Accuracy')
    plt.title(title) # Define o título do gráfico.
    plt.xlabel('Epochs') # Rótulo do eixo X.
    plt.ylabel('Accuracy') # Rótulo do eixo Y.
    plt.legend() # Mostra a legenda das linhas.
    plt.grid() # Adiciona uma grade ao gráfico.
    plt.show() # Exibe o gráfico.

# Chama a função para plotar os resultados combinados.
plot_history(history_tl, history_ft, "Transfer Learning + Fine-Tuning - VGG16")